# Objectif du notebook :
le code suivant a pour but de tester la lecture d'un fichier et d'en extraire les questions et propositions de réponses après sa conversion en images .jpeg.
Le modèle utilisé est 

In [16]:
#Installation de pdf2Image
pip install pdf2image
#Installation de groq
pip install groq

SyntaxError: invalid syntax (1265830268.py, line 2)

In [22]:
from pdf2image import convert_from_path
from groq import Groq
import base64
import os

### 1 - On convertit le pdf en images plates.

In [21]:
#Création du dossier spécifique pour stocker les images temporaires : 
import os
if(os.path.exists('data/images')==False):
    os.mkdir('data/images')

In [20]:
# Conversion du PDF en images
nomfichier = "data/pu_p01_aap04.pdf"
images = convert_from_path(nomfichier)

# Enregistre chacunes des pages comme une image
for i in range(len(images)):
    images[i].save('data/images/page' + str(i) + '.jpg', 'JPEG')

### 2 - On lit le document sur les serveurs Groq avec le LLM llama-4-scout-17b-16e-instruct

Lecture avec deux questions basiques, c'est à dire : "Describe this page" et "Extract the questions from this page"

In [39]:
#Clef Groq :
KEY="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"

In [40]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Récupération des pages du pdf :
pages = os.listdir('data/images/')

for page in pages : 
    image_path = "data/images/"+page
    base64_image = encode_image(image_path)
    client = Groq(api_key=KEY)
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Describe this page"},
                    {"type": "text", "text": "Extract the questions from this page"},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        model="meta-llama/llama-4-scout-17b-16e-instruct",
    )
    print("PAGE : "+page)
    print(chat_completion.choices[0].message.content)
    print()

PAGE : page0.jpg
The image presents a project proposal template for the BIMP-EAGA-ROK Cooperation Fund (BKCF), which is a cover page. The template is designed to guide applicants in submitting their project proposals.

**Extracted Questions:**

1. What is the organization name?
2. What type of organization are you (public or private)?
3. What is the organization's address?
4. Who is the focal person for the project, and what is their position?
5. What is the telephone number?
6. What is the email address?
7. What is the project title?
8. Which target country(ies) does the project focus on? (Brunei Darussalam, Malaysia, Indonesia, Philippines)
9. What is the target geographic location(s) of the project? (BIMP-EAGA or not)
10. What is the project duration? (1 year or 2 years)
11. What is the proposed project start date?
12. What is the proposed project completion date?
13. What is the proposed project budget (total in USD)?
14. Is there potential project co-financing (optional)?
15. What

Lecture avec une question plus 'complexe' à modifier en fonction des derniers échanges

In [41]:
for page in pages : 
    image_path = "data/images/"+page
    base64_image = encode_image(image_path)
    client = Groq(api_key=KEY)
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Extract all the questions of the page in a Python list format, including the answer choices when available."},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        model="meta-llama/llama-4-scout-17b-16e-instruct",
    )
    print("PAGE : "+page)
    print(chat_completion.choices[0].message.content)
    print()

PAGE : page0.jpg
Here is the list of questions extracted from the page in Python list format, including the answer choices when available:

```python
questions = [
    {
        "question": "Organization Name",
        "type": "text"
    },
    {
        "question": "Organization Type",
        "type": "multiple choice",
        "options": [
            "Public (e.g. NGOs/Research Institutions)",
            "Private"
        ]
    },
    {
        "question": "Organization Address",
        "type": "text"
    },
    {
        "question": "Focal Person Name / Position",
        "type": "text"
    },
    {
        "question": "Telephone Number",
        "type": "text"
    },
    {
        "question": "Email Address",
        "type": "text"
    },
    {
        "question": "Project Title",
        "type": "text"
    },
    {
        "question": "Target Country(ies)",
        "type": "multiple choice",
        "options": [
            "Brunei Darussalam",
            "Malaysia",
         